In [1]:
# File types: CSV, Json, XML
import glob 
import pandas as pd 
# This package helps us extract XML data
import xml.etree.ElementTree as ET 
from datetime import datetime 

In [22]:
# This file will store the log
log_file = "log_file.txt" 

# This file will store the data
target_file = "transformed_data.csv" 


def extract_csv(filename):
    df = pd.read_csv(filename, low_memory = False)
    return df

def extract_json(filename):
    df = pd.read_json(filename, lines = True)
    return df

def extract_xml(filename):
    df = pd.DataFrame(columns=['car_model','year_of_manufacture', 'price','fuel'])
    tree = ET.parse(filename)
    root = tree.getroot()
    for car in root:
        car_model = car.find('car_model').text
        year_of_manufacture = car.find('year_of_manufacture').text
        price = float(car.find('price').text)
        fuel = car.find('fuel').text
        df = pd.concat([df, pd.DataFrame([{'car_model':car_model, 'year_of_manufacture':year_of_manufacture, 'price':price, 'fuel':fuel}])])
    
    return df

def extract():
    extracted_data = pd.DataFrame(columns=['car_model', 'year_of_manufacture','price','fuel'])
    
    # process all csv files
    for csvfile in glob.glob('*csv'):
        extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_csv(csvfile))], ignore_index = True)
    
    for jsonfile in glob.glob('*.json'):
        extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_json(jsonfile))], ignore_index = True)
    
    for xmlfile in glob.glob('*.xml'):
        extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_xml(xmlfile))], ignore_index = True)
    
    return extracted_data

def transform(df):
    df['price'] = round(df.price,2) 
    return df

def load_data(target_file, clean_data):
    clean_data.to_csv(target_file)
    
def log_progress(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open(log_file, 'a') as f:
        f.write(timestamp + ',' + message + '\n')

In [19]:
# Log the initialization of the ETL process 
log_progress("ETL Job Started") 
print("ETL Job Started")

ETL Job Started


In [20]:
# Log the beginning of the Extraction process 
log_progress("Extract phase Started") 
print("Extract phase Started")
extracted_data = extract() 

Extract phase Started


In [23]:
# Log the beginning of the Transformation process 
log_progress("Transform phase Started") 
transformed_data = transform(extracted_data) 
print("Transformed Data") 
print(transformed_data) 

Transformed Data
        car_model year_of_manufacture     price    fuel
0            ritz                2014   5000.00  Petrol
1             sx4                2013   7089.55  Diesel
2            ciaz                2017  10820.90  Petrol
3         wagon r                2011   4253.73  Petrol
4           swift                2014   6865.67  Diesel
..            ...                 ...       ...     ...
85          camry                2006   3731.34  Petrol
86   land cruiser                2010  52238.81  Diesel
87  corolla altis                2012   8805.97  Petrol
88     etios liva                2013   5149.25  Petrol
89        etios g                2014   7089.55  Petrol

[90 rows x 4 columns]


In [25]:
# Log the completion of the Transformation process 
log_progress("Transform phase Ended") 
 
# Log the beginning of the Loading process 
log_progress("Load phase Started") 
load_data(target_file,transformed_data) 
 
# Log the completion of the Loading process 
log_progress("Load phase Ended") 
 
# Log the completion of the ETL process 
log_progress("ETL Job Ended") 